In [72]:
import pandas as pd

df = pd.read_excel("/home/jui/thesis-code/data/credit_card_clients.xls")

# Display the first few rows of the dataframe to verify it loaded correctly
print(df.head())


   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1      20000    2          2         1   24      2      2     -1     -1   
1   2     120000    2          2         2   26     -1      2      0      0   
2   3      90000    2          2         2   34      0      0      0      0   
3   4      50000    2          2         1   37      0      0      0      0   
4   5      50000    1          2         1   57     -1      0     -1      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...          0          0          0         0       689         0   
1  ...       3272       3455       3261         0      1000      1000   
2  ...      14331      14948      15549      1518      1500      1000   
3  ...      28314      28959      29547      2000      2019      1200   
4  ...      20940      19146      19131      2000     36681     10000   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default payment next month  
0         0         0   

In [73]:
df['EDUCATION'] = df['EDUCATION'].replace({1: 1, 2: 1, 3: 2, 4: 3})
print(df.head())

   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1      20000    2          1         1   24      2      2     -1     -1   
1   2     120000    2          1         2   26     -1      2      0      0   
2   3      90000    2          1         2   34      0      0      0      0   
3   4      50000    2          1         1   37      0      0      0      0   
4   5      50000    1          1         1   57     -1      0     -1      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...          0          0          0         0       689         0   
1  ...       3272       3455       3261         0      1000      1000   
2  ...      14331      14948      15549      1518      1500      1000   
3  ...      28314      28959      29547      2000      2019      1200   
4  ...      20940      19146      19131      2000     36681     10000   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default payment next month  
0         0         0   

In [74]:
# Count occurrences of each unique value across all PAY_* columns
value_counts = df[['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']].stack().value_counts()
# List of columns to modify
pay_columns = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

# Replace -2 with -1
df[pay_columns] = df[pay_columns].replace({-2:-1,0:-1})

# Count occurrences of each unique value across PAY_* columns
value_counts = df[pay_columns].stack().value_counts()

# Print result
print(value_counts)


-1    154974
 2     18964
 1      3722
 3      1430
 4       453
 7       218
 5       137
 6        74
 8        28
dtype: int64


In [76]:
df['MARRIAGE'] = df['MARRIAGE'].replace({0: 3})

In [77]:
df = df.drop(columns=['ID'])

In [78]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import xgboost as xgb
from packaging import version
import sklearn
from sklearn.metrics import accuracy_score, classification_report

def split_label(dataset, target_feature):
    X = dataset.drop([target_feature], axis=1)
    y = dataset[[target_feature]]
    return X, y

# Handle different scikit-learn versions for OneHotEncoder parameters
if version.parse(sklearn.__version__) < version.parse('1.2'):
    ohe_params = {"sparse": False}
else:
    ohe_params = {"sparse_output": False}

def create_classification_pipeline(X):
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    num_pipe = Pipeline([ 
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', **ohe_params))
    ])
    feat_pipe = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])

    # Using XGBClassifier with GPU support
    pipeline = Pipeline(steps=[('preprocessor', feat_pipe),
                            ('model', xgb.XGBClassifier(
                                tree_method='hist',  # Use hist for GPU training
                                random_state=0,
                                n_jobs=-1  # Use all available CPU cores for pre-processing
                            ))])

    return pipeline

In [79]:
target_feature = 'default payment next month'
categorical_features = []

# Split data into features and target
X, y = split_label(df, target_feature)

# Split data into train and test sets (80% training, 20% testing)
X_train_og, X_test_og, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=40)

# Create classification pipeline
pipeline = create_classification_pipeline(X_train_og)

# Convert target labels to numpy arrays for fitting
y_train = y_train[target_feature].to_numpy()
y_test = y_test[target_feature].to_numpy()

# Train the model using the pipeline
model = pipeline.fit(X_train_og, y_train)

y_pred = pipeline.predict(X_test_og)
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{class_report}\n")

Accuracy: 0.8190666666666667
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.94      0.89      5860
           1       0.64      0.39      0.48      1640

    accuracy                           0.82      7500
   macro avg       0.74      0.66      0.69      7500
weighted avg       0.80      0.82      0.80      7500




In [80]:
print(df[['SEX', 'LIMIT_BAL']].sample(5))

       SEX  LIMIT_BAL
552      1      50000
6007     2     190000
22195    2      70000
28640    2     150000
23154    2     180000


In [81]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
2025-02-11 14:38:12.567828: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739313492.581806  696626 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739313492.586078  696626 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-11 14:38:12.600376: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [82]:
from responsibleai.feature_metadata import FeatureMetadata
# Set up feature metadata for RAIInsights
feature_metadata = FeatureMetadata(categorical_features=categorical_features, dropped_features=[])

# Add the target feature back to the datasets
X_train_og_with_target = X_train_og.copy()
X_train_og_with_target[target_feature] = y_train

X_test_og_with_target = X_test_og.copy()
X_test_og_with_target[target_feature] = y_test
X_test_og_with_target = X_test_og_with_target.sample(n=1000, random_state=10)

In [83]:
for feature in categorical_features:
    print(f"Unique values in {feature} for training data: {X_train_og[feature].unique()}")
    print(f"Unique values in {feature} for test data: {X_test_og[feature].unique()}")


In [84]:
# Now, pass these modified DataFrames to RAIInsights
rai_insights = RAIInsights(model, X_train_og_with_target, X_test_og_with_target, target_feature, 'classification', feature_metadata=feature_metadata)

In [85]:
# Interpretability
rai_insights.explainer.add()
# Error Analysis
rai_insights.error_analysis.add()
# Counterfactuals: accepts total number of counterfactuals to generate, the label that they should have, and a list of 
                # strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=10, desired_class='opposite')


In [86]:
# Compute: Perform all tasks (this remains CPU-bound)
rai_insights.compute()

Causal Effects
Current Status: Generating Causal Effects.
Current Status: Finished generating causal effects.
Time taken: 0.0 min 5.7864002883434296e-05 sec
Counterfactual
Current Status: Generating 10 counterfactuals for 1000 samples


100%|██████████| 1000/1000 [20:16<00:00,  1.22s/it] 


Current Status: Generated 10 counterfactuals for 1000 samples.
Time taken: 20.0 min 28.316757647087798 sec
Error Analysis
Current Status: Generating error analysis reports.
Current Status: Finished generating error analysis reports.
Time taken: 0.0 min 0.14137085783295333 sec
Explanations
Current Status: Explaining 23 features
Current Status: Explained 23 features.
Time taken: 0.0 min 0.360141946002841 sec


In [87]:
ResponsibleAIDashboard(rai_insights)

ResponsibleAI started at http://localhost:8705
